In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import GroupKFold
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import optuna


/Users/mihaid/opt/anaconda3/envs/phytree/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class DynamicRegressionModel(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(DynamicRegressionModel, self).__init__()
        layers = [nn.Linear(input_size, hidden_sizes[0]), nn.ReLU()]
        
        for i in range(1, len(hidden_sizes)):
            layers += [nn.Linear(hidden_sizes[i-1], hidden_sizes[i]), nn.ReLU()]
        
        layers += [nn.Linear(hidden_sizes[-1], output_size)]
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)


In [3]:
COLS_TO_DROP = ['prune_name','rgft_name','orig_ds_ll','ll_prune','time_prune',
                'd_ll_prune', 'orig_ds_id','ll_rgft','time_rgft', 'd_ll_merged',                 
                'name2ntaxa_pruned_prune','name2ntaxa_remaining_prune',
                'name2ntaxa_pruned_rgft','name2ntaxa_remaining_rgft']

#COLS_TO_DROP = ['prune_name','rgft_name','orig_ds_ll','ll_prune','time_prune',
#                'd_ll_prune', 'orig_ds_id','ll_rgft','time_rgft', 'd_ll_merged']

TARGET_COL = 'd_ll_merged'

file_path = './data/validation_data/learning_all_moves.csv'
df = pd.read_csv(file_path, index_col='iteration', low_memory=False)


In [4]:
groups = df['orig_ds_id']
print(f'Are present {len(groups.unique())} different datasets')
data = df.drop(COLS_TO_DROP, axis=1)

labels = df[TARGET_COL].values
features = data.values

# Normalize features
scaler = StandardScaler()
#scaler = RobustScaler()
features_scaled = scaler.fit_transform(features)

# Convert to PyTorch tensors
features_tensor = torch.tensor(features_scaled, dtype=torch.float32)
labels_tensor = torch.tensor(labels, dtype=torch.float32)

Are present 266 different datasets


In [9]:
BATCH_SIZE = 64
OUTPUT_SIZE = 1  # predicting only 'd_ll_merged' value
NUM_EPOCHS = 10

def objective(trial):
	# Define the hyperparameters to tune
	lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
	num_layers = trial.suggest_int('num_layers', 1, 5)
	hidden_sizes = [trial.suggest_int(f'n_units_l{i}', 16, 128) for i in range(num_layers)]
	#n_splits = trial.suggest_int('n_splits', 5, len(groups.unique()))

	# Cross-validation
	gkf = GroupKFold(n_splits=10)
	mae_kfold = []
	r2_kfold = []
	for fold, (train_idx, val_idx) in enumerate(gkf.split(features_scaled, labels, groups=groups)):
		print(f"Start Fold {fold+1}")

		X_train_fold, y_train_fold = features_scaled[train_idx], labels[train_idx]
		X_val_fold, y_val_fold = features_scaled[val_idx], labels[val_idx]
		
		train_dataset = TensorDataset(torch.tensor(X_train_fold, dtype=torch.float32), 
									torch.tensor(y_train_fold, dtype=torch.float32))
		val_dataset = TensorDataset(torch.tensor(X_val_fold, dtype=torch.float32), 
									torch.tensor(y_val_fold, dtype=torch.float32))
		
		train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
		val_loader = DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE)
  
		input_size = X_train_fold.shape[1]  # Number of features
		model = DynamicRegressionModel(input_size, hidden_sizes, OUTPUT_SIZE)
		criterion = nn.MSELoss()
		optimizer = torch.optim.Adam(model.parameters(), lr=lr)		
		
		# Training loop for the current fold
		num_epochs = NUM_EPOCHS
		for epoch in range(num_epochs):
			epoch_losses = []
			for inputs, targets in train_loader:
				optimizer.zero_grad()
				outputs = model(inputs)
				loss = criterion(outputs, targets.view(-1, 1))
				loss.backward()
				optimizer.step()
				
				epoch_losses.append(loss.item())
			
			avg_loss = sum(epoch_losses) / len(epoch_losses)
			print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, AVGLoss: {avg_loss:.4f}')

		model.eval()  # Set the model to evaluation mode
		with torch.no_grad():
			predictions = []
			actuals = []
			for X_batch, y_batch in val_loader:
				y_pred = model(X_batch)
				predictions.append(y_pred.numpy())
				actuals.append(y_batch.numpy())

			# Flatten the list if predictions and actuals are nested lists
			predictions = np.concatenate(predictions, axis=0)
			actuals = np.concatenate(actuals, axis=0)

			mae = mean_absolute_error(actuals, predictions)
			mse = mean_squared_error(actuals, predictions)
			r2 = r2_score(actuals, predictions)
			print(f'Fold {fold+1}, MAE: {mae}, MSE: {mse}, R^2: {r2}')
		mae_kfold.append(mae)
		r2_kfold.append(r2)

	return np.mean(r2_kfold)

study = optuna.create_study(direction= 'maximize')
#study = optuna.create_study(directions=['minimize','maximize'])
study.optimize(objective, n_trials=30)

[I 2024-02-17 00:40:48,288] A new study created in memory with name: no-name-c68298de-dd09-4758-aded-657823b7e7b0


/Users/mihaid/opt/anaconda3/envs/phytree/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/mihaid/opt/anaconda3/envs/phytree/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


Start Fold 1
Epoch [1/10], Loss: 7400.6753, AVGLoss: 28014.3609
Epoch [2/10], Loss: 14990.5625, AVGLoss: 21731.5352
Epoch [3/10], Loss: 11205.0771, AVGLoss: 17229.5196
Epoch [4/10], Loss: 14147.7373, AVGLoss: 16079.6509
Epoch [5/10], Loss: 6761.5991, AVGLoss: 15158.7718
Epoch [6/10], Loss: 46965.9609, AVGLoss: 14321.9484
Epoch [7/10], Loss: 11324.2051, AVGLoss: 13579.5589
Epoch [8/10], Loss: 19528.2441, AVGLoss: 12993.6143
Epoch [9/10], Loss: 4097.0659, AVGLoss: 12565.6541
Epoch [10/10], Loss: 21152.4922, AVGLoss: 12208.5294
Fold 1, MAE: 93.67870330810547, MSE: 26042.904296875, R^2: 0.32972757236738404
Start Fold 2
Epoch [1/10], Loss: 26112.5254, AVGLoss: 22308.5251
Epoch [2/10], Loss: 3407.9590, AVGLoss: 16241.8058
Epoch [3/10], Loss: 9133.9580, AVGLoss: 14536.7019
Epoch [4/10], Loss: 10329.0547, AVGLoss: 14112.1211
Epoch [5/10], Loss: 4473.9487, AVGLoss: 13809.7138
Epoch [6/10], Loss: 9549.0742, AVGLoss: 13550.8261
Epoch [7/10], Loss: 17834.3613, AVGLoss: 13363.2113
Epoch [8/10], Los

[I 2024-02-17 01:14:54,668] Trial 0 finished with value: 0.3867656384835983 and parameters: {'lr': 7.047249111653818e-05, 'num_layers': 4, 'n_units_l0': 76, 'n_units_l1': 54, 'n_units_l2': 104, 'n_units_l3': 33}. Best is trial 0 with value: 0.3867656384835983.


Fold 10, MAE: 69.723876953125, MSE: 15845.6806640625, R^2: 0.5613878985633882
Start Fold 1


...

Fold 10, MAE: 72.32888793945312, MSE: 15488.669921875, R^2: 0.5712700146056426
Start Fold 1


/Users/mihaid/opt/anaconda3/envs/phytree/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/mihaid/opt/anaconda3/envs/phytree/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


Epoch [1/10], Loss: 19411.2930, AVGLoss: 31536.0268
Epoch [2/10], Loss: 25182.1680, AVGLoss: 26510.2512
Epoch [3/10], Loss: 19318.4141, AVGLoss: 25264.9635
Epoch [4/10], Loss: 21267.2207, AVGLoss: 23960.0873
Epoch [5/10], Loss: 10027.9297, AVGLoss: 22438.4050
Epoch [6/10], Loss: 18341.2441, AVGLoss: 20720.6018
Epoch [7/10], Loss: 8798.7842, AVGLoss: 19095.2771
Epoch [8/10], Loss: 18660.0820, AVGLoss: 17990.5390
Epoch [9/10], Loss: 6937.2905, AVGLoss: 17377.9368
Epoch [10/10], Loss: 11380.5742, AVGLoss: 16975.5280
Fold 1, MAE: 89.35684204101562, MSE: 22223.9609375, R^2: 0.4280165857472584
Start Fold 2
Epoch [1/10], Loss: 48699.8047, AVGLoss: 26178.6820
Epoch [2/10], Loss: 9073.4023, AVGLoss: 21757.9683
Epoch [3/10], Loss: 8561.2090, AVGLoss: 21082.3218
Epoch [4/10], Loss: 13929.4551, AVGLoss: 20346.1652
Epoch [5/10], Loss: 12506.0410, AVGLoss: 19492.2390
Epoch [6/10], Loss: 7982.6455, AVGLoss: 18380.5643
Epoch [7/10], Loss: 11446.7051, AVGLoss: 17062.9651
Epoch [8/10], Loss: 19931.4609,

[I 2024-02-17 17:07:02,921] Trial 29 finished with value: 0.40263376615399044 and parameters: {'lr': 4.902399951952506e-05, 'num_layers': 3, 'n_units_l0': 24, 'n_units_l1': 91, 'n_units_l2': 48}. Best is trial 28 with value: 0.41893336317808016.


Fold 10, MAE: 75.07625579833984, MSE: 16798.365234375, R^2: 0.5350173952940345


In [ ]:
# Use the best hyperparameters
best_hyperparams = study.best_trial
print(f"Best hyperparameters: {best_hyperparams.params}")

In [23]:
import pickle

study_name = "valStudy"  # Unique identifier of the study.
storage_name = "sqlite:///{}.db".format(study_name)
# Save the sampler with pickle to be loaded later.
with open("sampler.pkl", "wb") as fout:
    pickle.dump(study.sampler, fout)

restored_sampler = pickle.load(open("sampler.pkl", "rb"))
study = optuna.create_study(
    study_name=study_name, storage=storage_name, load_if_exists=True, sampler=restored_sampler
)

[I 2024-02-17 17:49:49,526] Using an existing study with name 'valStudy30Trial' instead of creating a new one.


In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
fig = optuna.visualization.plot_param_importances(study)
fig.show()